# Deprecated

In [ ]:
import json
import requests
import urllib
import requests
from json import loads
import pandas as pd

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
X_ELS_APIKey = os.getenv("X_ELS_APIKey")
X_ELS_Insttoken = os.getenv("X_ELS_Insttoken")

In [ ]:
keyword = "%28cancer+of+the+prostate%29+%28+molecular+targeted+therapy%29"

## Scopus

Pontos que precisaremos trabalhar -

- aceita no máximo 25 artigos por query
- https://dev.elsevier.com/documentation/ScopusSearchAPI.wadl
- Precisamos usar uma outra chave de api disponibilizada pelo teste online usando CORS 
https://dev.elsevier.com/search.html#/Scopus_Search

In [ ]:
url = f"http://api.elsevier.com/content/search/scopus?query=KEY{keyword}"

# Make a request for a given paper
resp = requests.get(
    url,
    headers={
        "Accept": "application/json",
        "X-ELS-APIKey": X_ELS_APIKey,
        "X-ELS-Insttoken": X_ELS_Insttoken,
        "X-RateLimit-Reset": None,
    },
)

# Check the status of the request
print(resp.status_code)

# Check if the problem is the weekly download limit (usually 10000)
try:
    print(resp.header["X-RateLimit-Remaining"])
except:
    pass

In [ ]:
dic = loads(resp.content)

data = pd.DataFrame.from_dict(dic)

for i in data.iloc[0]:
    scopus_data = pd.DataFrame(i)

scopus_data.head()

## Science Direct

Pontos que precisaremos trabalhar
- aceita no máximo 100 artigos por query
- https://dev.elsevier.com/documentation/ScienceDirectSearchAPI.wadl
- Precisamos usar uma outra chave de api disponibilizada pelo teste online usanod CORS 
https://dev.elsevier.com/search.html#/ScienceDirect_Search_V2

In [ ]:
url = f"https://api.elsevier.com/content/search/sciencedirect?query={keyword}&count=100"

# Make a request for a given paper
resp = requests.get(
    url,
    headers={
        "Accept": "application/json",
        "X-ELS-APIKey": X_ELS_APIKey,
        "X-ELS-Insttoken": X_ELS_Insttoken,
        "X-RateLimit-Reset": None,
    },
)

# Check the status of the request
print(resp.status_code)

# Check if the problem is the weekly download limit (usually 10000)
try:
    print(resp.header["X-RateLimit-Remaining"])
except:
    pass

In [ ]:
dic = loads(resp.content)

data = pd.DataFrame.from_dict(dic)

for i in data.iloc[0]:
    scidir_data = pd.DataFrame(i)

scidir_data.columns

# Version 2

In [4]:
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import json
import pandas as pd

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
apikey = os.getenv("X_ELS_APIKey")
insttoken = os.getenv("X_ELS_Insttoken")
keyword = "(cancer of the prostate) AND (molecular targeted therapy)"

In [3]:
client = ElsClient(apikey)
client.inst_token = insttoken

In [8]:
doc_srch = ElsSearch("cancer prostata", "sciencedirect")
t = doc_srch.execute(client, get_all=True)
print("doc_srch has", len(doc_srch.results), "results.")

doc_srch has 5000 results.


In [10]:
abstract = []
pubtype = []

for i in doc_srch.results_df['prism:doi']:
    doi_doc = FullDoc(doi = i)
    if doi_doc.read(client):
        abstract.append(doi_doc.data['coredata']['dc:description'])
        pubtype.append(doi_doc.data['coredata']['pubType'])    
        #doi_doc.write()   
    else:
        print ("Read document failed.")
        
doc_srch.results_df['abstract'] = abstract
doc_srch.results_df['pubtype'] = pubtype

In [11]:
doc_srch.results_df.head(3)

,@_fa,load-date,link,dc:identifier,prism:url,dc:title,dc:creator,prism:publicationName,prism:coverDate,prism:doi,openaccess,pii,authors,prism:volume,prism:startingPage,prism:endingPage,abstract,pubtype
0,true,2023-02-03T00:00:00.000Z,{'self': 'https://api.elsevier.com/content/art...,DOI:10.1016/j.rx.2022.12.005,https://api.elsevier.com/content/article/pii/S...,"Resonancia magnética en la detección, estadifi...",J. C. Vilanova,Radiología,2023-02-03,10.1016/j.rx.2022.12.005,False,S0033833823000097,"{'author': [{'$': 'J. C. Vilanova'}, {'$': 'V....",NaN,NaN,NaN,\n El cáncer de próstata es m...,fla
1,true,2023-03-24T00:00:00.000Z,{'self': 'https://api.elsevier.com/content/art...,DOI:10.1016/j.acuro.2023.03.003,https://api.elsevier.com/content/article/pii/S...,Impacto de la realización de resonancia magnét...,D. Vieira e Brito,Actas Urológicas Españolas,2023-03-24,10.1016/j.acuro.2023.03.003,False,S0210480623000542,"{'author': [{'$': 'D. Vieira e Brito'}, {'$': ...",NaN,NaN,NaN,\n Introducción\n ...,fla
2,true,2022-10-30T00:00:00.000Z,{'self': 'https://api.elsevier.com/content/art...,DOI:10.1016/S1761-3310(22)47080-0,https://api.elsevier.com/content/article/pii/S...,"Técnicas, indicaciones y resultados del vaciam...",C. Laine,EMC - Urología,2022-12-31,10.1016/S1761-3310(22)47080-0,False,S1761331022470800,"{'author': [{'$': 'C. Laine'}, {'$': 'F. Rozet...",54,1,6,\n El vaciamiento ganglionar ...,rev


In [18]:
doc_srch.results_df.to_csv('../../data/sci_dir_data.csv')

## Scopus

In [ ]:
doc_srch_scopus = ElsSearch("cancer prostata", "scopus")
t = doc_srch_scopus.execute(client, get_all=True)
print("doc_srch has", len(doc_srch_scopus.results), "results.")

In [17]:
doc_srch_scopus.results_df.to_csv('../../data/scopus_data.csv')

In [6]:
doc_srch_scopus = pd.read_csv('../../data/scopus_data.csv')

In [ ]:
dicts = {}

for i in doc_srch_scopus['prism:url']:
    scp_doc = AbsDoc(uri = i)
    if scp_doc.read(client):
        if 'dc:description' in scp_doc.data['coredata']:
            dicts[i] = scp_doc.data['coredata']['dc:description']
        else:
            dicts[i] = 'None'
    else:
        dicts[i] = 'Failed'

In [62]:
len(abstract)

5000

In [ ]:
doc_srch_scopus.results_df['abstract'] = abstract

## Conferência

In [ ]:
doi_doc = FullDoc(doi = '10.1016/j.acuro.2023.03.011')
if doi_doc.read(client):
    print ("doi_doc.title: ", doi_doc.title)
    doi_doc.write()   
else:
    print ("Read document failed.")

In [ ]:
doi_doc.data